In [2]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Note: Need to add these files locally before running
income_file = '17zpallnoagi.csv'
mh_file = 'MarketHealthIndex_Zip.csv'

df_income = pd.read_csv(income_file)
df_mh = pd.read_csv(mh_file, encoding='unicode_escape')

print(df_income.isnull().sum())

print(df_mh.isnull().sum())

STATEFIPS    0
STATE        0
ZIPCODE      0
AGI_STUB     0
N1           0
            ..
A11900       0
N11902       0
A11902       0
N12000       0
A12000       0
Length: 153, dtype: int64
RegionType                  0
RegionName                  0
City                        0
State                       0
Metro                     600
CBSATitle                 600
SizeRank                14089
MarketHealthIndex           0
SellForGain              4609
PrevForeclosed          11544
ForeclosureRatio        10836
ZHVI                      130
MoM                       130
YoY                       130
ForecastYoYPctChange     2743
StockOfREOs             14089
NegativeEquity            395
Delinquency               395
DaysOnMarket              127
dtype: int64


In [9]:
# Drop rows with null values for ZHVI
df_mh = df_mh.dropna(subset=['ZHVI'])
print(df_mh.isnull().sum())

RegionType                  0
RegionName                  0
City                        0
State                       0
Metro                     598
CBSATitle                 598
SizeRank                13959
MarketHealthIndex           0
SellForGain              4609
PrevForeclosed          11496
ForeclosureRatio        10812
ZHVI                        0
MoM                         0
YoY                         0
ForecastYoYPctChange     2613
StockOfREOs             13959
NegativeEquity            395
Delinquency               395
DaysOnMarket              127
dtype: int64
